¿Que hace este script?
* Calcula para cada dispositivo de red: trafico.totales.traficoIN, trafico.totales.traficoOUT', trafico.totales.consumoGB'.
* Calcula para cada site_id: trafico.totales.sitio.traficoIN, trafico.totales.sitio.traficoOUT, trafico.totales.sitio.consumoGB

In [56]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [57]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [58]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy

Estos valores se deben ajustar según ambiente. No es automático ya que no hay separación de ambientes

In [59]:
indice = parametros.conteo_centros_index
indice_control = parametros.tableros_mintic_control

### Función para generar JSON compatible con ES

In [60]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISO, entre otros).

In [61]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud','COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
    
    datos_semilla = pd.DataFrame([x["_source"] for x in elastic_docs])
    
except:
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()
    

Se valida latitud y longitud, se genera campo location y se renombran los campos de semilla

In [62]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'
    
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['centros.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['centros.location']=datos_semilla['centros.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

datos_semilla = datos_semilla.rename(columns={'nombre_municipio': 'centros.nombreMunicipio'
                                              , 'nombre_departamento' : 'centros.nombreDepartamento'
                                              , 'nombre_centro_pob': 'centros.localidad'
                                              , 'nombreSede' : 'centros.nomCentroDigital'
                                              , 'energiadesc' : 'centros.sistemaEnergia'
                                              , 'COD_ISO' : 'centros.codISO'
                                              , 'id_Beneficiario' : 'centros.idBeneficiario'})
datos_semilla.fillna('', inplace=True)

In [77]:
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["centros.location"]=='')].index)

In [78]:
datos_semilla

,centros.nomCentroDigital,centros.codISO,centros.localidad,site_id,centros.nombreDepartamento,centros.sistemaEnergia,centros.nombreMunicipio,centros.idBeneficiario,centros.location
0,I. E. SAN ANTONIO DE PIOJÓ SEDE LOS OLIVOS,CO-ATL,LOS OLIVOS,22514-ZZZY767,ATLÁNTICO,RED INTERCONECTADA,PIOJÓ,22514,"10.039416,-73.20525"
1,I.E. MARTILLO - SEDE PRINCIPAL,CO-ATL,KRA 4 NO 2A-15,22519-ZZZY558,ATLÁNTICO,SIN INFORMACIÓN,PONEDERA,22519,"6.99780241,-75.59734752"
2,I.E. TECNICA AGROPECUARIA SAN CAYETANO DE GALL...,CO-ATL,CRA. 5 N. 5 - 58,22538-ZZZY571,ATLÁNTICO,SIN INFORMACIÓN,SABANALARGA,22538,"10.83546085,-75.045541840"
3,I.E. TÉCNICA JUAN V PADILLA,CO-ATL,VEREDA EL VAIVEN,11130-ZZZY429,ATLÁNTICO,SIN INFORMACIÓN,JUAN DE ACOSTA,11130,"10.79876297,-74.92334914"
4,I.E. TECNICA MARIA INMACULADA DE PITAL DE MEGUA,CO-ATL,KR 15 11 96,74366-ZGYO061,ATLÁNTICO,SIN INFORMACIÓN,BARANOA,74366,"4.710277778,-75.01111111"
...,...,...,...,...,...,...,...,...,...
6942,ESCUELA SANTA ISABEL,CO-VAU,COMUNIDAD SANTA ISABEL,,VAUPÉS,SOLUCIÓN SOLAR,PACOA,67733,"6.39026335,-73.138349330"
6943,ESCUELA RURAL DE SAN LUIS DE PIEDRA ÑI,CO-VAU,COMUNIDAD DE SAN LUIS,,VAUPÉS,SIN INFORMACIÓN,PACOA,70622,"0.08164385,-70.07416384"
6944,ESCUELA RURAL DE SANTA ROSA,CO-VAU,COMUNIDAD DE SANTA ROSA,,VAUPÉS,SIN INFORMACIÓN,PACOA,70623,"4.596389,-74.074639"
6945,ESCUELA RURAL DE TOAKA,CO-VAU,COMUNIDAD DE TOAKA,,VAUPÉS,SIN INFORMACIÓN,PACOA,70624,"1.007644476,-71.25103723"


### leyendo indice cambium-devicedevices

Se lee la información de los dispositivos de red monitoreados por Cambium. En esta lectura no hay referencia de fechas ya que solo hay una ocurrencia por MAC de dispositivo de red.

* site_id es la llave para cruzar con cada centro de conexión.
* mac, IP y name son datos básicos del dispositivo.
* ap_group identifica los dispositivos como INDOOR u OUTDOOR

In [79]:
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac",'name',"ip","ap_group"]
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     datos_dev = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
    
    datos_dev = pd.DataFrame([x["_source"] for x in elastic_docs])
    
    
except:
    exit()

Se descartan registros con site_id vacios y se limpian los NaN del dataframe

In [80]:
datos_dev.dropna(subset=['site_id'])
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)
datos_dev.sort_values(['site_id','ap_group'], inplace=True)

Se limpian datos mal formados de ap_group

In [81]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

In [82]:
datos_dev = datos_dev.drop_duplicates('mac')

Se renombran campos según formato del indice final

In [83]:
datos_dev = datos_dev.rename(columns={'ap_mac' : 'centros.macRed','ap_group': 'centros.apGroup'
                                        , 'ip': 'centros.IP'
                                        , 'mac' : 'centros.macRed'
                                        , 'name' : 'centros.deviceName'})

In [84]:
datos_dev

,centros.apGroup,centros.IP,centros.deviceName,site_id,centros.macRed
3325,INDOOR,172.25.41.219,32094-ZZZY229-EL-BRILLANTE-AP-INT,10003700,BC:E6:7C:5D:20:E1
3326,OUTDOOR,172.25.41.220,32094-ZZZY229-EL-BRILLANTE-AP-EXT2,10003700,BC:A9:93:0E:2F:15
3327,OUTDOOR,172.25.41.221,32094-ZZZY229-EL-BRILLANTE-AP-EXT1,10003700,BC:A9:93:0E:63:3E
3288,INDOOR,172.25.46.251,10501-ZZZY338-AP-INT,10501-ZZZY338,BC:E6:7C:ED:8A:FE
3289,OUTDOOR,172.25.46.253,10501-ZZZY338-AP-EXT2,10501-ZZZY338,BC:E6:7C:E8:3F:AE
...,...,...,...,...,...
1043,OUTDOOR,172.25.132.77,PRUEBA-LABORATORIO-AP-EXT2,PRUEBA-LABORATORIO,BC:E6:7C:E7:FC:2B
1044,OUTDOOR,172.25.132.76,PRUEBA-LABORATORIO-AP-EXT1,PRUEBA-LABORATORIO,BC:E6:7C:E7:FC:21
1426,INDOOR,172.25.117.11,47354-ZGYO835-AP-INT,TUMBATORO,BC:E6:7C:5F:3A:F1
1427,OUTDOOR,172.25.117.12,47354-ZGYO835-AP-EXT1,TUMBATORO,BC:E6:7C:E8:3F:8A


### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [85]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ["conteos.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-conteos"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['tablero06.fechaControl']
except:
    fecha_ejecucion = '2021-05-01 00:00:00'
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-05-01 00:00:00


## Se lee la información de cambium device performance

 Se toma los valores de dispositivos de red y su desempeño.
 * mac del dispositivo de red
 * timestamp es la fecha y hora de la medición
 * radio.* volumen de datos descargados(r) y cargados(t)

In [86]:
def traePerformance(fecha_max,fecha_tope):
    total_docs = 5000000
    response = es.search(
        index= parametros.cambium_d_p_index,
        body={
                "_source": ["mac","timestamp","radio.5ghz.rx_bps",
                           "radio.5ghz.tx_bps","radio.24ghz.rx_bps"
                          ,"radio.24ghz.tx_bps"]
              , "query": {
                  "range": {
                    "timestamp": {
                      "gte": fecha_max,
                      "lt": fecha_tope
                    }
                  }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

    return pd.DataFrame([x["_source"] for x in elastic_docs])

### Lanzando  ejecución de consulta

* Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [73]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_performance = traePerformance(fecha_max_mintic,fecha_tope_mintic)

if datos_performance is None or datos_performance.empty:
    while (datos_performance is None or datos_performance.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        datos_performance = traePerformance(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

In [74]:
#datos_performance = traePerformance('2021-06-10 09:00:00','2021-06-10 10:00:00')

funcion para insertar en indice: 

In [75]:
use_these_keys = ['centro.nomCentroDigital',
                  'centro.codISO',
                  'centro.localidad',
                  'centro.siteID',
                  'centro.nombreDepartamento',
                  'centro.sistemaEnergia',
                  'centro.nombreMunicipio',
                  'centro.idBeneficiario',
                  'centro.apGroup',
                  'centro.IP',
                  'centro.deviceName',
                  'centro.macRed',
                  'totales.centro.fechaControl',
                  'totales.centro.traficoIN',
                  'totales.centro.traficoOUT',
                  'totales.centro.consumoGB',
                  'totales.centro.fecha',
                  'totales.centro.anyo',
                  'totales.centro.mes',
                  'totales.centro.dia',
                  'totales.centro.hora',
                  'totales.centro.minuto',
                  'nombreDepartamento',
                    'nombreMunicipio',
                    'idBeneficiario',
                    'fecha',
                    'anyo',
                    'mes',
                    'dia',
                  '@timestamp']
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'consumo-' + document['centro.siteID'] + '-' + document['centro.macRed'] + '-' +document['totales.centro.fechaControl']}",
                "_source": filterKeys(document),
            }

# Insertando consumo a indice 

* Calculo de indicadores de consumos para cada AP. 
* Se toman los valores de rx y tx como descarga y carga
* de datos_dev se toma el site_id y el ap_group

In [76]:

try:
    datos_performance['fecha_control'] = datos_performance["timestamp"].str[0:-4] + '0:00'
    datos_performance.rename(columns={'mac': 'centro.macRed'}, inplace=True)
    datos_performance.replace('','0',inplace=True)
    datos_performance.fillna({'radio.5ghz.rx_bps':0, 'radio.5ghz.tx_bps':0,
                      'radio.24ghz.rx_bps':0, 'radio.24ghz.tx_bps':0 },inplace=True)
    datos_performance[['radio.5ghz.rx_bps','radio.5ghz.tx_bps','radio.24ghz.rx_bps','radio.24ghz.tx_bps']] = datos_performance[['radio.5ghz.rx_bps','radio.5ghz.tx_bps','radio.24ghz.rx_bps','radio.24ghz.tx_bps']].astype(int)

    aux_performance=datos_performance[['centro.macRed','fecha_control'
                                       ,'radio.5ghz.rx_bps'
                                       ,'radio.5ghz.tx_bps'
                                       ,'radio.24ghz.rx_bps'
                                       ,'radio.24ghz.tx_bps']].groupby(['centro.macRed','fecha_control']).agg(['sum']).reset_index()
    aux_performance.columns = aux_performance.columns.droplevel(1)
    aux_performance['totales.centro.traficoIN_aux'] = aux_performance['radio.5ghz.rx_bps'] + aux_performance['radio.24ghz.rx_bps']
    aux_performance['totales.centro.traficoOUT_aux'] = aux_performance['radio.5ghz.tx_bps'] + aux_performance['radio.24ghz.tx_bps']
    
    aux_performance = pd.merge(aux_performance, datos_dev, on='centro.macRed',how='inner')
    mintic_02 = pd.merge(datos_semilla,  aux_performance, on='site_id',how='inner')
    
    #La información de tráfico se convierte a GigaBytes y se toman 6 decimales
    mintic_02['totales.centro.traficoIN'] = round((mintic_02['totales.centro.traficoIN_aux']/float(1<<30)),6)
    mintic_02['totales.centro.traficoOUT'] = round((mintic_02['totales.centro.traficoOUT_aux']/float(1<<30)),6)
    mintic_02['totales.centro.consumoGB'] = mintic_02['totales.centro.traficoIN'] + mintic_02['totales.centro.traficoOUT']
    #Se totaliza entrante y saliente
    mintic_02['totales.centro.consumoGB'] = round(mintic_02['totales.centro.consumoGB'],6)

    ### Generando columnas con fecha, anyo, mes, dia, hora y minuto por separado
    mintic_02["totales.centro.fecha"] = mintic_02["fecha_control"].str.split(" ", n = 1, expand = True)[0]
    mintic_02["totales.centro.hora"] = mintic_02["fecha_control"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    mintic_02["totales.centro.minuto"] = mintic_02["fecha_control"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    mintic_02["totales.centro.anyo"] = mintic_02["totales.centro.fecha"].str[0:4]
    mintic_02["totales.centro.mes"] = mintic_02["totales.centro.fecha"].str[5:7]
    mintic_02["totales.centro.dia"] = mintic_02["totales.centro.fecha"].str[8:10]
    ### Renombrado de campos
    mintic_02.rename(columns={'site_id': 'centro.siteID'
                             ,'fecha_control' : 'totales.centro.fechaControl'}, inplace=True)

    ##nulos a cero
    mintic_02.fillna({'totales.centro.consumoGB':0,
                      'totales.centro.traficoIN':0,
                      'totales.centro.traficoOUT':0
                      },inplace=True)
    #cambia valores a tipo float
    mintic_02[['totales.centro.consumoGB','totales.centro.traficoIN','totales.centro.traficoOUT']] = mintic_02[['totales.centro.consumoGB','totales.centro.traficoIN','totales.centro.traficoOUT']].astype(float)
    
    mintic_02['nombreDepartamento'] = mintic_02['centro.nombreDepartamento']
    mintic_02['nombreMunicipio'] = mintic_02['centro.nombreMunicipio']
    mintic_02['idBeneficiario'] = mintic_02['centro.idBeneficiario']
    mintic_02['fecha'] = mintic_02['totales.centro.fecha']
    mintic_02['anyo'] = mintic_02['totales.centro.anyo']
    mintic_02['mes'] = mintic_02['totales.centro.mes']
    mintic_02['dia'] = mintic_02['totales.centro.dia']
    mintic_02['@timestamp'] = now.isoformat()
    salida = helpers.bulk(es, doc_generator(mintic_02))
    print("Fecha: ", now,"- Datos Trafico Consumos en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No se insertaron datos de consumos en indice principal")

Fecha:  2021-07-29 13:46:29.405156 - No se insertaron datos de consumos en indice principal


## En otra  jerarquía se escriben los dispositivos conectados

Se toma el dataframe del proceso anterior para realizar el siguiente proceso:
* Se cuentan la cantidad de dispositivos WAN/LAN (OUTDOOR/INDOOR), agrupando por fecha y sitio
* Cantidad dispositivos WAN/LAN conectados se calcula validando cuando son OUTDOOR/INDOOR y tienen trafico(traficoOUT)
* Cantidad de dispositivos desconectados, restando los dos anteriores

Datos generados
* trafico.totales.cantDevWAN
* trafico.totales.cantDevLAN
* trafico.totales.cantDevConectadosWAN
* trafico.totales.cantDevDesconectadosWAN
* trafico.totales.cantDevConectadosLAN
* trafico.totales.cantDevDesconectadosLAN
* trafico.totales.cantDev

In [48]:
try:
    mintic_03 = mintic_02[['centro.nomCentroDigital',
                      'centro.codISO',
                      'centro.localidad',
                      'centro.siteID',
                      'centro.nombreDepartamento',
                      'centro.sistemaEnergia',
                      'centro.nombreMunicipio',
                      'centro.idBeneficiario',
                      'centro.location',
                      'centro.apGroup',
                      'totales.centro.fechaControl',
                      'totales.centro.fecha',
                      'totales.centro.anyo',
                      'totales.centro.mes',
                      'totales.centro.dia',
                      'totales.centro.hora',
                      'totales.centro.minuto',
                      'totales.centro.traficoIN',
                      'totales.centro.traficoOUT',
                      'totales.centro.consumoGB',]].groupby(['centro.nomCentroDigital',
                                                              'centro.codISO',
                                                              'centro.localidad',
                                                              'centro.siteID',
                                                              'centro.nombreDepartamento',
                                                              'centro.sistemaEnergia',
                                                              'centro.nombreMunicipio',
                                                              'centro.idBeneficiario',
                                                              'totales.centro.fechaControl',
                                                              'totales.centro.fecha',
                                                              'totales.centro.anyo',
                                                              'totales.centro.mes',
                                                              'totales.centro.dia',
                                                              'totales.centro.hora',
                                                              'totales.centro.minuto']).agg(['sum']).reset_index()
    mintic_03.columns = mintic_03.columns.droplevel(1)
    mintic_03.rename(columns={'totales.centro.traficoIN' : 'totales.sitio.centro.traficoIN'
                          ,'totales.centro.traficoOUT' : 'totales.sitio.centro.traficoOUT'
                          ,'totales.centro.consumoGB' : 'totales.sitio.centro.consumoGB'
                         }, inplace=True)
except:
    pass

Funcion para insertar en indice la cantidad de dispositivos conectados 
* la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán


In [49]:
use_these_keys = ['centro.nomCentroDigital',
                  'centro.codISO',
                  'centro.localidad',
                  'centro.siteID',
                  'centro.nombreDepartamento',
                  'centro.sistemaEnergia',
                  'centro.nombreMunicipio',
                  'centro.idBeneficiario',
                  'centro.location',
                  'centro.apGroup',
                  'totales.centro.fechaControl',
                  'totales.centro.cantDevWAN',
                  'totales.centro.cantDevConectadosWAN',
                  'totales.centro.cantDevDesconectadosWAN',
                  'totales.centro.cantDevLAN',
                  'totales.centro.cantDevConectadosLAN',
                  'totales.centro.cantDevDesconectadosLAN',
                  'totales.centro.cantDev',
                  'totales.centro.cantDevConectados',
                  'totales.centro.cantDevDesconectados',
                  'totales.centro.fecha',
                  'totales.centro.anyo',
                  'totales.centro.mes',
                  'totales.centro.dia',
                  'totales.centro.hora',
                  'totales.centro.minuto',
                  'totales.sitio.centro.traficoIN',
                  'totales.sitio.centro.traficoOUT',
                  'totales.sitio.centro.consumoGB',
                  '@timestamp']

def doc_generator_dis(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'Conectados-' + document['centro.siteID'] + '-' +document['totales.centro.fechaControl']}",
                "_source": filterKeys(document),
            }


# Insertando 

In [50]:
try:
    cantDevWAN = mintic_02[(mintic_02['centro.apGroup']=='OUTDOOR')][['totales.centro.fechaControl','centro.macRed','centro.siteID']].groupby(['centro.siteID','totales.centro.fechaControl'])['centro.macRed'].nunique().reset_index()    
    cantDevWAN.rename(columns={'centro.macRed': 'totales.centro.cantDevWAN'}, inplace=True)
    mintic_03 = pd.merge(mintic_03,  cantDevWAN, on=['centro.siteID','totales.centro.fechaControl'],how='left')

    cantDevConectadosWAN = mintic_02[(mintic_02['totales.centro.traficoOUT']>0) & (mintic_02['centro.apGroup']=='OUTDOOR')][['totales.centro.fechaControl','centro.macRed','centro.siteID']].groupby(['centro.siteID','totales.centro.fechaControl'])['centro.macRed'].nunique().reset_index()
    cantDevConectadosWAN.rename(columns={'centro.macRed': 'totales.centro.cantDevConectadosWAN'}, inplace=True)
    mintic_03 = pd.merge(mintic_03, cantDevConectadosWAN, on=['centro.siteID','totales.centro.fechaControl'],how='left')
    mintic_03.fillna({'totales.centro.cantDevWAN':0,
                      'totales.centro.cantDevConectadosWAN':0
                      },inplace=True)
    mintic_03['totales.centro.cantDevDesconectadosWAN'] = mintic_03['totales.centro.cantDevWAN'] - mintic_03['totales.centro.cantDevConectadosWAN']

    #La misma lógica se aplica para calcular las cantidades para dispositivos LAN, pero filtrando los INDOOR
    cantDevLAN = mintic_02[(mintic_02['centro.apGroup']=='INDOOR')][['totales.centro.fechaControl','centro.macRed','centro.siteID']].groupby(['centro.siteID','totales.centro.fechaControl'])['centro.macRed'].nunique().reset_index()
    cantDevLAN.rename(columns={'centro.macRed': 'totales.centro.cantDevLAN'}, inplace=True)
    mintic_03 = pd.merge(mintic_03,  cantDevLAN, on=['centro.siteID','totales.centro.fechaControl'],how='left')

    cantDevConectadosLAN = mintic_02[~(mintic_02['totales.centro.traficoIN']>0) & (mintic_02['centro.apGroup']=='INDOOR')][['totales.centro.fechaControl','centro.macRed','centro.siteID']].groupby(['centro.siteID','totales.centro.fechaControl'])['centro.macRed'].nunique().reset_index()
    cantDevConectadosLAN.rename(columns={'centro.macRed': 'totales.centro.cantDevConectadosLAN'}, inplace=True)

    mintic_03 = pd.merge(mintic_03, cantDevConectadosLAN, on=['centro.siteID','totales.centro.fechaControl'],how='left')
    mintic_03.fillna({'totales.centro.cantDevLAN':0,
                      'totales.centro.cantDevConectadosLAN':0
                      },inplace=True)
    mintic_03['totales.centro.cantDevDesconectadosLAN'] = mintic_03['totales.centro.cantDevLAN'] - mintic_03['totales.centro.cantDevConectadosLAN']

    mintic_03.fillna({'totales.centro.cantDevConectadosWAN':0,
                      'totales.centro.cantDevDesconectadosWAN':0,
                      'totales.centro.cantDevConectadosLAN':0,
                      'totales.centro.cantDevDesconectadosLAN':0,
                      'totales.centro.sitio.traficoIN':0,
                      'totales.centro.sitio.traficoOUT':0,
                      'totales.centro.sitio.consumoGB':0
                      },inplace=True)
    mintic_03['totales.centro.cantDev'] = mintic_03['totales.centro.cantDevLAN'] + mintic_03['totales.centro.cantDevWAN']
    mintic_03[['totales.centro.cantDev','totales.centro.cantDevConectadosWAN','totales.centro.cantDevDesconectadosWAN','totales.centro.cantDevConectadosLAN','totales.centro.cantDevDesconectadosLAN']] = mintic_03[['totales.centro.cantDev','totales.centro.cantDevConectadosWAN','totales.centro.cantDevDesconectadosWAN','totales.centro.cantDevConectadosLAN','totales.centro.cantDevDesconectadosLAN']].astype(int)

    mintic_03['totales.centro.cantDevConectados'] = mintic_03['totales.centro.cantDevConectadosWAN'] + mintic_03['totales.centro.cantDevConectadosLAN']
    mintic_03['totales.centro.cantDevDesconectados'] = mintic_03['totales.centro.cantDevDesconectadosLAN'] + mintic_03['totales.centro.cantDevDesconectadosWAN']
    mintic_03[['totales.centro.cantDevConectados'
              ,'totales.centro.cantDevDesconectados'
              ,'totales.centro.cantDevWAN'
              ,'totales.centro.cantDevLAN']] = mintic_03[['totales.centro.cantDevConectados'
                                                          ,'totales.centro.cantDevDesconectados'
                                                          ,'totales.centro.cantDevWAN'
                                                          ,'totales.centro.cantDevLAN']].astype(int)    
    mintic_03['@timestamp'] = now.isoformat()
    salida = helpers.bulk(es, doc_generator_dis(mintic_03))
    print("Fecha: ", now,"- Datos Dispositivos Conectados en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No se insertaron datos de dispositivos conectados en indice principal")




Fecha:  2021-07-29 13:43:29.079575 - No se insertaron datos de dispositivos conectados en indice principal


### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [51]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia-tablero06',
        body = { 'jerarquia-tablero06': 'jerarquia-tablero06','tablero06.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-05-01 00:10:00
